In [1]:
import pandas as pd
import transformers
import duckdb
import time 
import os
import torch

In [2]:

x = torch.rand(5, 3)
print(x)

tensor([[0.1918, 0.5972, 0.3064],
        [0.3669, 0.8368, 0.7654],
        [0.3418, 0.5756, 0.1250],
        [0.7443, 0.4716, 0.3364],
        [0.9476, 0.0175, 0.5428]])
